In [ ]:
from planetaryimage import PDS3Image
from matplotlib import pylab as plt
import numpy as np
import matplotlib.patches as patches
import requests
from bs4 import BeautifulSoup
import os
import zipfile
import io
import alphashape
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.geometry import Point
import shutil
from shapely.errors import TopologicalError
import math
import re
import rasterio
from pathlib import Path

In [ ]:
# Parámetros y configuración inicial
volumenes = [35, 45, 65, 82, 86, 87, 93, 98, 100, 101, 108, 111, 120, 126, 127, 131, 149, 157, 161, 166, 167, 174, 177, 181, 189, 193, 195, 199, 200, 201, 203, 209, 210, 211, 218, 220, 229, 234, 239, 240, 243, 248, 250, 253, 261, 265, 271, 276, 277, 285]
output_dir = Path(r"C:/Users/DELL/Desktop/IAFE/pds_img")
os.makedirs(output_dir, exist_ok=True)
sampling=100
keywords = ["BIXQI"]
path= "C:/Users/DELL/Desktop/IAFE/pds_img"

In [ ]:
def descargar_y_extraer_zip(url, destino):
    try:
        print(f"Descargando {url} ...")
        respuesta = requests.get(url,verify = False)
        
        if respuesta.status_code != 200:
            print(f"Error al descargar {url} (status: {respuesta.status_code}).")
            return

        # Leer el contenido del ZIP en memoria y extraerlo en `destino`
        with zipfile.ZipFile(io.BytesIO(respuesta.content)) as z:
            z.extractall(destino)
            print(f"Archivos extraídos en {destino}")

    except Exception as e:
        print(f"Error inesperado al procesar {url}: {e}")

In [ ]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def explorar_volumen(volumenes, output_dir, keywords):
    for volumen_actual in volumenes:
        if volumen_actual < 100:
            base_url = f"https://planetarydata.jpl.nasa.gov/img/data/cassini/cassini_orbiter/CORADR_00{volumen_actual}/DATA/BIDR/"
        else:
            base_url = f"https://planetarydata.jpl.nasa.gov/img/data/cassini/cassini_orbiter/CORADR_0{volumen_actual}/DATA/BIDR/"

        volumen_dir = os.path.join(output_dir, f"volumen_{volumen_actual}")
        print(f"Explorando volumen {volumen_actual}...")

        try:
            respuesta = requests.get(base_url, verify=False)
            if respuesta.status_code != 200:
                print(f"❌ No se pudo acceder al volumen {volumen_actual} (status: {respuesta.status_code})")
                continue

            soup = BeautifulSoup(respuesta.content, 'html.parser')
            enlaces = soup.find_all('a', href=True)

            for keyword in keywords:
                zip_encontrado = False
                for enlace in enlaces:
                    href = enlace['href']
                    if href.endswith('.ZIP') and keyword in href:
                        url_zip = base_url + href
                        descargar_y_extraer_zip(url_zip, volumen_dir)
                        zip_encontrado = True
                        print(f"✅ Archivo '{href}' descargado para la palabra clave '{keyword}'.")
                        break

                if not zip_encontrado:
                    print(f"⚠️ No se encontró archivo ZIP con la palabra clave '{keyword}' en el volumen {volumen_actual}.")

        except requests.exceptions.RequestException as e:
            print(f"❌ Error de conexión en {base_url}: {e}")
        except Exception as e:
            print(f"❌ Error al procesar {base_url}: {e}")

In [ ]:
explorar_volumen(volumenes,output_dir,keywords)

In [ ]:
def matriz_incidencia(Flyby):
    carpeta = Path(output_dir) / f"volumen_{Flyby}"
    archivos = list(carpeta.glob("*.IMG"))     #lista archivos en la carpeta

    for archivo in archivos:
        if archivo.name.startswith("BIE"):
                img = PDS3Image.open(str(archivo))
                matriz_incidencia = img.image

                np.save(carpeta/"matriz_incidencia.npy", matriz_incidencia)
                return matriz_incidencia

In [ ]:
def matriz_latitud(Flyby):
    carpeta = Path(output_dir) / f"volumen_{Flyby}"
    archivos = list(carpeta.glob("*.IMG"))     #lista archivos en la carpeta

    for archivo in archivos:
        if archivo.name.startswith("BIN"):
                img = PDS3Image.open(str(archivo))
                matriz_latitud = img.image

                np.save(carpeta/"matriz_latitud.npy", matriz_latitud)
                return matriz_latitud

In [ ]:
def matriz_longitud(Flyby):
    carpeta = Path(output_dir) / f"volumen_{Flyby}"
    archivos = list(carpeta.glob("*.IMG"))     #lista archivos en la carpeta

    for archivo in archivos:
        if archivo.name.startswith("BIT"):
                img = PDS3Image.open(str(archivo))
                matriz_longitud = img.image


                np.save(carpeta/"matriz_longitud.npy", matriz_longitud)
                return matriz_longitud

In [ ]:
def matriz_sigma(Flyby):
    carpeta = Path(output_dir) / f"volumen_{Flyby}"
    archivos = list(carpeta.glob("*.IMG"))     #lista archivos en la carpeta

    for archivo in archivos:
        if archivo.name.startswith("BIX"):
                img = PDS3Image.open(str(archivo))
                matriz_sigma = img.image

                np.save(carpeta/"matriz_sigma.npy", matriz_sigma)
                return matriz_sigma

In [ ]:
def guardar_matrices(volumenes):
    for volumen in volumenes:
        matriz_latitud(volumen)
        matriz_longitud(volumen)
        matriz_sigma(volumen)
        matriz_incidencia(volumen)

guardar_matrices(volumenes)

In [ ]:
def procesar_incidencia(Flyby, sampling=100):         
    carpeta_destino = f"{output_dir}/volumen_{Flyby}"
    img_dict = {"inc": None}
    
    # Leer archivos en la carpeta
    for archivo in os.listdir(carpeta_destino):
        if archivo.startswith("BIE"):  # Ajustar según el prefijo del archivo
            img_dict["inc"] = PDS3Image.open(os.path.join(carpeta_destino, archivo))
    
    # Validar que se encontró el archivo de incidencia
    if img_dict["inc"] is None:
        raise FileNotFoundError("No se encontró el archivo de ángulos de incidencia en la carpeta.")
    
    # Extraer datos de incidencia
    inc = img_dict["inc"].image[::sampling, ::sampling].flatten()  # Reducir datos con sampling
    inc_validos = inc[inc > 0]  # Filtrar valores inválidos como ceros o negativos
    
    if len(inc_validos) == 0:
        raise ValueError("No hay valores válidos de incidencia en los datos.")
    
    # Calcular el promedio de ángulos de incidencia válidos
    promedio = np.mean(inc_validos)
    print(f"Promedio de ángulo de incidencia: {promedio:.2f} grados")
    return promedio

In [ ]:
def poda_por_incidencia(volumenes):
    lista_poda_por_incidencia = []

    for i in range(len(volumenes)):
        try:
            inc1 = procesar_incidencia(volumenes[i])
        except Exception as e:
            print(f"Error procesando incidencia para {volumenes[i]}: {e}")
            continue  # saltar a la siguiente i

        for j in range(i+1, len(volumenes)):
            try:
                inc2 = procesar_incidencia(volumenes[j])
            except Exception as e:
                print(f"Error procesando incidencia para {volumenes[j]}: {e}")
                continue  # saltar a la siguiente j

            if 20 <= abs(inc1 - inc2) <= 30:
                lista_poda_por_incidencia.append(
                    (volumenes[i], volumenes[j], abs(inc1 - inc2))
                )
    archivo_salida = os.path.join(output_dir, "lista_poda_incidencia.txt")
    with open(archivo_salida, "w") as f:
        for elemento in lista_poda_por_incidencia:
            f.write(f"{elemento}\n")

    return lista_poda_por_incidencia
print(poda_por_incidencia(volumenes))

In [ ]:
with open("lista_poda_incidencia.txt", "r") as f:
    lista = []
    for line in f:
        line = line.strip().replace("(", "").replace(")", "").replace("'", "")
        partes = line.split(",")
        if len(partes) == 3:
            v1, v2, dif = partes
            lista.append((v1.strip(), v2.strip(), float(dif.strip())))
print(lista)

In [ ]:
def mascara_latitud(F1,F2):
    carpeta1=output_dir/f'volumen_{F1}'
    carpeta2= output_dir/f'volumen_{F2}'
    m_lat_1= np.load(carpeta1/"matriz_latitud.npy")
    m_lat_2= np.load(carpeta2/"matriz_latitud.npy")
    mascara_latitud_F1 = np.isin(m_lat_1, m_lat_2).astype(int)
    mascara_latitud_F2 = np.isin(m_lat_2, m_lat_1).astype(int)
    return mascara_latitud_F1,mascara_latitud_F2

In [ ]:
def mascara_longitud(F1,F2):
    carpeta1=output_dir/f'volumen_{F1}'
    carpeta2= output_dir/f'volumen_{F2}'
    m_lon_1= np.load(carpeta1/"matriz_longitud.npy")
    m_lon_2= np.load(carpeta2/"matriz_longitud.npy")
    mascara_longitud_F1 = np.isin(m_lon_1, m_lon_2).astype(int)
    mascara_longitud_F2 = np.isin(m_lon_2, m_lon_1).astype(int)
    return mascara_longitud_F1,mascara_longitud_F2

In [ ]:
def mascara_interseccion(F1,F2):
    m_lon_1,m_lon_2 = mascara_longitud(F1,F2)
    m_lat_1,m_lat_2 = mascara_latitud(F1,F2)
    mascara_interseccion_1= m_lat_1 * m_lon_1
    mascara_interseccion_2= m_lon_2 * m_lat_2
    return mascara_interseccion_1,mascara_interseccion_2

In [ ]:
def interseccion(lista):
    for tupla in lista:
        F1=tupla[0]
        F2=tupla[1]
        mascara_interseccion_1, mascara_interseccion_2 = mascara_interseccion(F1, F2)

        if np.any(mascara_interseccion_1 == 1):
            carpeta_des = os.path.join(output_dir/f"interseccion_{F1}_{F2}")
            os.makedirs(carpeta_des, exist_ok=True)

            np.save(os.path.join(carpeta_des, "mascara_interseccion_1.npy"), mascara_interseccion_1)
            np.save(os.path.join(carpeta_des, "mascara_interseccion_2.npy"), mascara_interseccion_2)

            print(f"Máscaras guardadas en: {carpeta_des}")

interseccion(lista)
     

In [ ]:
def mascara_aplicada(lista,matrices_a_aplicar=["incidencia","latitud", "longitud", "sigma"]):
    for tupla in lista:
        F1=tupla[0]
        F2=tupla[1]
        carpeta_interseccion = output_dir / f"interseccion_{F1}_{F2}"
        if carpeta_interseccion.exists():
            print(f"Procesando intersección: {F1}-{F2}")
                
                # Cargar máscaras
            try:
                m1 = np.load(carpeta_interseccion / "mascara_interseccion_1.npy")
                m2 = np.load(carpeta_interseccion / "mascara_interseccion_2.npy")
            except Exception as e:
                print(f"Error cargando máscaras para {F1}-{F2}: {e}")
                continue
            for nombre_matriz in matrices_a_aplicar:
                    try:
                        matriz1 = np.load(output_dir / f"volumen_{F1}" / f"matriz_{nombre_matriz}.npy")
                        matriz2 = np.load(output_dir / f"volumen_{F2}" / f"matriz_{nombre_matriz}.npy")

                        resultado1 = matriz1 * m1
                        resultado2 = matriz2 * m2

                        # Guardar resultados aplicados
                        np.save(carpeta_interseccion / f"{nombre_matriz}_F1_con_mascara.npy", resultado1)
                        np.save(carpeta_interseccion / f"{nombre_matriz}_F2_con_mascara.npy", resultado2)
                        
                    except Exception as e:
                        print(f"Error procesando {nombre_matriz} para {F1} o {F2}: {e}")

In [ ]:
mascara_aplicada(lista)

In [ ]:
def graficar_matriz(path_archivo, titulo=None, cmap="viridis"):
    matriz = np.load(path_archivo)
    plt.figure(figsize=(8, 6))
    plt.imshow(matriz, cmap=cmap)
    plt.colorbar(label='Valor')
    plt.title(titulo if titulo else str(path_archivo))
    plt.xlabel('Columnas')
    plt.ylabel('Filas')
    plt.tight_layout()
    plt.show()

graficar_matriz("C:/Users/DELL/Desktop/IAFE/pds_img/interseccion_82_174/latitud_F1_con_mascara.npy")
graficar_matriz("C:/Users/DELL/Desktop/IAFE/pds_img/interseccion_82_174/latitud_F2_con_mascara.npy")

In [ ]:
def graficar_lat_lon(lat_path, lon_path, mascara_path=None, titulo="Huella geográfica", color='blue'):
    lat = np.load(lat_path)
    lon = np.load(lon_path)
    
    if mascara_path is not None:
        mascara = np.load(mascara_path)
        # Aplanamos todo y filtramos solo los puntos con máscara activa
        lat_flat = lat[mascara > 0]
        lon_flat = lon[mascara > 0]
    else:
        lat_flat = lat.flatten()
        lon_flat = lon.flatten()
    
    plt.figure(figsize=(8, 6))
    plt.scatter(lon_flat, lat_flat, s=0.5, c=color, alpha=0.6)
    plt.xlabel("Longitud")
    plt.ylabel("Latitud")
    plt.title(titulo)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
F1 = 82
interseccion_dir = output_dir / f"interseccion_{F1}_174"

graficar_lat_lon(
    lat_path = interseccion_dir / f"latitud_F1_con_mascara.npy",
    lon_path = interseccion_dir / f"longitud_F1_con_mascara.npy",
    mascara_path = interseccion_dir / "mascara_interseccion_1.npy",
    titulo = f"Flyby {F1} - Huella con máscara aplicada",
    color='green'
)


In [ ]:
F1 = 174
interseccion_dir = output_dir / f"interseccion_82_{F1}"

graficar_lat_lon(
    lat_path = interseccion_dir / f"latitud_F2_con_mascara.npy",
    lon_path = interseccion_dir / f"longitud_F2_con_mascara.npy",
    mascara_path = interseccion_dir / "mascara_interseccion_2.npy",
    titulo = f"Flyby {F1} - Huella con máscara aplicada",
    color='green')

In [ ]:
def graficar_interseccion_y_guardar(F1, F2, output_dir, matrices=["latitud", "longitud", "sigma", "incidencia"], cmap="viridis"):
    carpeta = output_dir / f"interseccion_{F1}_{F2}"
    carpeta_plots = carpeta / "plots"
    carpeta_plots.mkdir(exist_ok=True)

    for nombre in matrices:
        for sufijo in ["F1", "F2"]:
            path = carpeta / f"{nombre}_{sufijo}_con_mascara.npy"
            if path.exists():
                matriz = np.load(path)
                plt.figure(figsize=(6, 5))
                ax = plt.gca()
                ax.set_facecolor("white")  # Fondo del área del gráfico
                plt.imshow(matriz, cmap=cmap)
                plt.colorbar(label="Valor")
                titulo = f"{nombre.capitalize()} - {sufijo} - Intersección {F1}-{F2}"
                plt.title(titulo)
                plt.xlabel("Columnas")
                plt.ylabel("Filas")
                plt.tight_layout()
                nombre_archivo = carpeta_plots / f"{nombre}_{sufijo}.png"
                plt.savefig(nombre_archivo, facecolor='white')  # Fondo del archivo PNG
                plt.close()
                print(f"✅ Guardado: {nombre_archivo}")
            else:
                print(f"⚠️ No se encontró: {path}")


In [ ]:
for tupla in lista:
    F1 = int(tupla[0])
    F2 = int(tupla[1])
    carpeta = Path(output_dir) / f"interseccion_{F1}_{F2}"
    carpeta_plots = carpeta / "plots"
    
    if carpeta.exists():
        if carpeta_plots.exists():
            print(f"⏭️  Ya existe carpeta 'plots' para intersección_{F1}_{F2}, se salta.")
            continue  # ← CORRECTO: salta al siguiente elemento del loop

        graficar_interseccion_y_guardar(F1, F2, output_dir)
    else:
        print(f"❌ Carpeta interseccion_{F1}_{F2} no existe, se salta.")